In [22]:
# Imports
from pathlib import Path

import plotly.express as px
import polars as pl


In [35]:
transformed_data_with_nodes_path = Path("/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes")

sample_id_to_pop_path = Path("/home/jpatterson87/for_classes/gnn_for_diabetes/data/sample_info/sample_id_to_population.txt")

In [36]:
sample_id_to_pop = pl.read_csv(file=Path(sample_id_to_pop_path),
                               has_header=True,
                               sep="\t")

In [33]:
demographic_tabulation = (sample_id_to_pop
    .groupby(by="population_description")
    .agg([
        pl.count().alias("num_samples")
    ])
    .sort("num_samples")
)
demographics_plot = px.bar(x=demographic_tabulation.select("population_description").to_series(), 
                           y=demographic_tabulation.select("num_samples").to_series(),
                           title="Numbers of Samples for each Population",
                           labels={"x": "Population Description",
                                   "y": "Count"})
# https://stackoverflow.com/questions/68400851/how-to-rotate-xtick-label-bar-chart-plotly-express
demographics_plot.update_xaxes(tickangle=270)
demographics_plot.show()
demographics_plot.write_html(file="/home/jpatterson87/for_classes/gnn_for_diabetes/figures/demographics_plot.html")


In [47]:
# https://stackoverflow.com/questions/23782215/iterate-through-files
# Get a list of tuples containing files and their labels.
files_and_labels = []

for tsv in transformed_data_with_nodes_path.glob("*.tsv"):
    sample_id = tsv.name.split(".")[0]
    label = (sample_id_to_pop
        .filter(
            (pl.col("sample_id") == sample_id)
        )
        .select(
            pl.col("population_description")
        )
        # run unique just in case
        .unique()
        .to_series()
        .to_list()
    )
    if len(label) > 0:
        files_and_labels.append((tsv, label[0]))


[(PosixPath('/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes/HG01046.hard-filtered.sliced.vcf.tsv'), 'Chinese Dai in Xishuangbanna, China'), (PosixPath('/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes/HG01047.hard-filtered.sliced.vcf.tsv'), 'Puerto Rican in Puerto Rico'), (PosixPath('/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes/HG01048.hard-filtered.sliced.vcf.tsv'), 'Puerto Rican in Puerto Rico'), (PosixPath('/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes/HG01049.hard-filtered.sliced.vcf.tsv'), 'Puerto Rican in Puerto Rico'), (PosixPath('/home/jpatterson87/for_classes/gnn_for_diabetes/data/transformed_data_from_vms/with_nodes/HG01050.hard-filtered.sliced.vcf.tsv'), 'Puerto Rican in Puerto Rico')]
